In [1]:
import pandas as pd

df_music = pd.read_csv("raw_data/music_main_2025-04-30.csv")
df_gidamps = pd.read_csv("raw_data/gidamps_sampling_2025-04-30.csv")
df_music_demographics = pd.read_csv("raw_data/music_demographics_2025-04-30.csv")

In [2]:
df_music['cucq_date']# Convert cucq_date to datetime format first
df_music['cucq_date'] = pd.to_datetime(df_music['cucq_date']) 

# Filter for dates after Jan 1st 2025
cutoff_date = pd.to_datetime('2025-01-01')

music_future_dates = df_music[df_music['cucq_date'] > cutoff_date].shape[0]

# Convert sampling_date to datetime if not already done
df_gidamps['sampling_date'] = pd.to_datetime(df_gidamps['sampling_date'])

# Filter for non-NA cucq_5 and dates after cutoff
cutoff_date = pd.to_datetime('2025-01-01')
filtered_gidamps = df_gidamps[
    (df_gidamps['cucq_5'].notna()) & 
    (df_gidamps['sampling_date'] > cutoff_date)
]

print(f"Number of GIDAMPs rows with non-NA cucq_5 after Jan 1st 2025: {len(filtered_gidamps)}")

# Optional: Display first few rows to verify
print("\nFirst few rows of filtered data:")
print(filtered_gidamps[['sampling_date', 'cucq_5']].head())

print(f"Number of MUSIC rows after Jan 1st 2025: {music_future_dates}")

Number of GIDAMPs rows with non-NA cucq_5 after Jan 1st 2025: 1

First few rows of filtered data:
     sampling_date  cucq_5
1091    2025-02-03    14.0
Number of MUSIC rows after Jan 1st 2025: 38


In [3]:
df_music_existing = pd.read_csv("raw_data/gidamps_music_combined_fatigue_df_111224_with_cmh.csv")

# remove mid prefix from df_music['study_id']
df_music['study_id'] = df_music['study_id'].str.replace('MID-', '', regex=False)

In [4]:
music_future_dates = df_music[df_music['cucq_date'] > cutoff_date]

# check music_future_dates for study_ids in df_music_existing
existing_study_ids = df_music_existing['study_id'].unique()
music_future_study_ids = music_future_dates['study_id'].unique()
# Find study_ids in music_future_dates that are not in df_music_existing
missing_study_ids = set(music_future_study_ids) - set(existing_study_ids)
print(f"Number of unique study_ids in music_future_dates not in df_music_existing: {len(missing_study_ids)}")
print(f"Missing study_ids: {missing_study_ids}")

Number of unique study_ids in music_future_dates not in df_music_existing: 9
Missing study_ids: {'90-121', '92-26', '91-54', '92-34', '92-33', '92-35', '92-28', '91-52', '191-2'}


In [5]:
# subset music_futures_dates to only include the missing study_ids

music_future_dates_missing = music_future_dates[
    music_future_dates['study_id'].isin(missing_study_ids)
]

We could potentially add 10 more datapoints in from GI-DAMPs and MUSIC recruited after 2025-01-01.

In [6]:
# Extract cols from filtered_gidamps and music_future_dates_missing

cols_to_extract = [
    "cucq_5",
    "age",
    "sex", 
    "height",
    "weight",
    "smoking_status",
    "study_group",
    "date_of_diagnosis",
    "montreal_cd_location",
    "montreal_cd_behaviour",
    "montreal_upper_gi",
    "montreal_perianal",
    "montreal_uc_extent",
    "montreal_uc_severity",
    "has_active_symptoms",
    "haemoglobin",
    "red_cell_count",
    "white_cell_count",
    "neutrophils",
    "lymphocytes",
    "monocytes",
    "eosinophils",
    "basophils",
    "platelets",
    "urea",
    "creatinine",
    "sodium",
    "potassium",
    "crp",
    "albumin",
    "calprotectin",
    "sampling_steroids",
    "sampling_abx",
    "sampling_asa",
    "sampling_aza",
    "sampling_mp",
    "sampling_ifx",
    "sampling_ada",
    "sampling_vedo",
    "sampling_uste",
    "sampling_tofa",
    "sampling_mtx",
    "sampling_ciclosporin",
    "sampling_filgo",
    "sampling_upa",
    "sampling_risa"
]

In [7]:
# Check that filtered_gidamps has all the columns
missing_cols = [col for col in cols_to_extract if col not in filtered_gidamps.columns]
if missing_cols:
    print(f"Missing columns in filtered_gidamps: {missing_cols}")
else:
    print("All columns are present in filtered_gidamps.")

All columns are present in filtered_gidamps.


In [8]:
# Check that music_future_dates_missing has all the columns
missing_cols = [col for col in cols_to_extract if col not in music_future_dates_missing.columns]
if missing_cols:
    print(f"Missing columns in music_future_dates_missing: {missing_cols}")
else:
    print("All columns are present in music_future_dates_missing.")

All columns are present in music_future_dates_missing.


90-121 and 191-2 to recollect data

In [9]:
backfill_df = df_music_demographics[['study_id','age', 'sex', 'study_group', 'date_of_diagnosis']].copy()

# Remove 'MID-' prefix from study_id in backfill_df
backfill_df['study_id'] = backfill_df['study_id'].str.replace('MID-', '', regex=False)

music_cols = ['study_id'] + cols_to_extract
music_output_df = music_future_dates_missing[music_cols].copy()

# Fill missing values in music_output_df from backfill_df
columns_to_backfill = ['age', 'sex', 'study_group', 'date_of_diagnosis']
for column in columns_to_backfill:
    # Create a mapping dictionary from backfill_df
    backfill_map = backfill_df.set_index('study_id')[column].to_dict()
    
    # Only fill values that are missing (NaN)
    mask = music_output_df[column].isna()
    music_output_df.loc[mask, column] = music_output_df.loc[mask, 'study_id'].map(backfill_map)

# Check if we successfully filled in the missing values
print("Number of missing values after backfill:")
for column in columns_to_backfill:
    missing_count = music_output_df[column].isna().sum()
    print(f"{column}: {missing_count}")

# Display sample of updated records
print("\nSample of updated records:")
music_output_df[['study_id'] + columns_to_backfill].head()

Number of missing values after backfill:
age: 0
sex: 0
study_group: 0
date_of_diagnosis: 0

Sample of updated records:


,study_id,age,sex,study_group,date_of_diagnosis
539,90-121,33.0,male,cd,2018-06-01
796,91-52,23.0,female,cd,2024-02-07
806,91-54,60.0,female,cd,1990-01-01
925,92-26,18.0,female,uc,2024-01-15
933,92-28,22.0,male,uc,2024-06-01


## Output DF

In [10]:
gidamps_cols = ['sampling_setting', 'study_id'] + cols_to_extract
gidamps_output_df = filtered_gidamps[gidamps_cols].copy() # single participant from endoscopy

In [11]:
gidamps_output_df.head()

,sampling_setting,study_id,cucq_5,age,sex,height,weight,smoking_status,study_group,date_of_diagnosis,...,sampling_ifx,sampling_ada,sampling_vedo,sampling_uste,sampling_tofa,sampling_mtx,sampling_ciclosporin,sampling_filgo,sampling_upa,sampling_risa
1091,endoscopy,GID-763,14.0,41,male,183.0,79.4,non_smoker,uc,2017-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
music_output_df.head()

,study_id,cucq_5,age,sex,height,weight,smoking_status,study_group,date_of_diagnosis,montreal_cd_location,...,sampling_ifx,sampling_ada,sampling_vedo,sampling_uste,sampling_tofa,sampling_mtx,sampling_ciclosporin,sampling_filgo,sampling_upa,sampling_risa
539,90-121,7.0,33.0,male,177.8,78.5,non_smoker,cd,2018-06-01,L2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796,91-52,3.0,23.0,female,165.0,103.6,non_smoker,cd,2024-02-07,L2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,91-54,14.0,60.0,female,155.0,49.9,ex_smoker,cd,1990-01-01,L3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,92-26,5.0,18.0,female,169.5,58.7,non_smoker,uc,2024-01-15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,92-28,0.0,22.0,male,188.0,82.0,non_smoker,uc,2024-06-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Concat the two dataframes
combined_output_df = pd.concat([gidamps_output_df, music_output_df], ignore_index=True)

# Drop the rows where haemoglobin is NaN
combined_output_df = combined_output_df.dropna(subset=['haemoglobin'])

combined_output_df.head()

,sampling_setting,study_id,cucq_5,age,sex,height,weight,smoking_status,study_group,date_of_diagnosis,...,sampling_ifx,sampling_ada,sampling_vedo,sampling_uste,sampling_tofa,sampling_mtx,sampling_ciclosporin,sampling_filgo,sampling_upa,sampling_risa
0,endoscopy,GID-763,14.0,41.0,male,183.0,79.4,non_smoker,uc,2017-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,91-52,3.0,23.0,female,165.0,103.6,non_smoker,cd,2024-02-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,91-54,14.0,60.0,female,155.0,49.9,ex_smoker,cd,1990-01-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,92-26,5.0,18.0,female,169.5,58.7,non_smoker,uc,2024-01-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,92-28,0.0,22.0,male,188.0,82.0,non_smoker,uc,2024-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Clean up output_df

# Capitalize df['sex'] first letter
combined_output_df['sex'] = combined_output_df['sex'].str.capitalize()
# Make study group all caps
combined_output_df['study_group'] = combined_output_df['study_group'].str.upper()
# Replace underscores with space in smoking_status and capitalize first letter
combined_output_df['smoking_status'] = combined_output_df['smoking_status'].str.replace('_', ' ')
combined_output_df['smoking_status'] = combined_output_df['smoking_status'].str.capitalize()
# Replace has_active_symptoms 0 with No and 1 with Yes
combined_output_df['has_active_symptoms'] = combined_output_df['has_active_symptoms'].replace({0: 'No', 1: 'Yes'})

In [16]:

combined_output_df.to_csv("raw_data/scotland_additional_validation_data.csv", index=False)